In [11]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import math
from PIL import Image,ImageOps
import cv2 as cv
import os

In [12]:
#Mean среднее значение
def getMean(hist,raz):
  mean=0
  for i in range (raz):
    mean+=(i+1)*hist[i]
  return mean

#Variance дисперсия
def getVariance(hist,raz,mean):
  variance=0
  for i in range (raz):
    variance+=pow((i+1)-mean,2)*hist[i]
  return variance


#Skewness коэффициент ассиметрии
def getSkewness(hist,raz,mean,variance):
  skewness=0
  for i in range (raz):
    skewness+=pow((i+1)-mean,3)*hist[i]
  return skewness/pow(variance,1.5)


#Kurtosis коэффициент эксцесса
def getKurtosis(hist,raz,mean,variance):
  kurtosis=0
  for i in range (raz):
    kurtosis+=pow((i+1)-mean,4)*hist[i]
  return (kurtosis/pow(variance,2)-3)


#Perc процентные характеристики гистограммы (1,10,50,90,99 процентов) (количество элементов гистограммы, сумма которых меньше заданного процента)
def getPerc(hist,raz,val):
  temp=0
  perc=0
  while temp<val:
    temp+=hist[perc]
    perc+=1
  return perc

In [13]:
def getHaralickFeatures(arr1,rows1,cols1,CCMP1,Px1,Py1,PxPy1,PxMy1,dx1,dy1,features1,start1):
  CoocurateMatrix(arr1,rows1,cols1,dx1,dy1,CCMP1) #рассчёт матрицы совпадений
  Asm=getAngularSecondMoment(CCMP1)
  features1[start1]=Asm
  contrast=getContrast(CCMP1)
  features1[start1+1]=contrast
  Calc_Px(CCMP1,Px1)
  Calc_Py(CCMP1,Py1)
  Mx = Calc_M(Px1)
  My = Calc_M(Py1)
  Dx = Calc_D(Px1, Mx)
  Dy = Calc_D(Py1, My)
  Calc_P_X_Plus_Y(CCMP1, PxPy1)
  Calc_P_X_Minus_Y(CCMP1, PxMy1)
  correl=getCorr(CCMP1,Mx,My,Dx,Dy)
  features1[start1+2]=correl
  sumofsq=GetSumOfSquares(CCMP1, Mx)
  features1[start1+3]=sumofsq
  invdifmom=GetInverseDifferenceMoment(CCMP1)
  features1[start1+4]=invdifmom
  avsum=GetSumAverage(PxPy1,qlc+qlc)
  features1[start1+5]=avsum
  entsum=GetSumEntropy(PxPy1,qlc+qlc)
  features1[start1+7]=entsum
  varsum=GetSumVariance(PxPy1,qlc+qlc,avsum)
  features1[start1+6]=varsum
  ent=GetEntropy(CCMP1)
  features1[start1+8]=ent
  diffvar=GetDiffereceVariance(PxMy1,qlc+qlc,entsum)
  features1[start1+9]=diffvar
  diffent=GetDefferenceEntropy(PxMy1,qlc+qlc)
  features1[start1+10]=diffent


def CoocurateMatrix(arr1,rows1,cols1,dx1,dy1,CCMP1):
  temp = [0]*qlc
  for i in range(qlc):
    temp[i]=[0]*qlc
  sum=0
  for i in range (abs(min(0,dy1)),(rows1-max(0,dy1))):
    for j in range (abs(min(0,dx1)),(cols1-max(0,dx1))):
      k=(arr1[i][j])
      l=(arr1[i+dy1][j+dx1])
      k=int(k/quantization_div)
      l=int(l/quantization_div)
      temp[k][l]+=1
      temp[l][k]+=1
      sum+=1
  for i in range (qlc):
    for j in range (qlc):
      CCMP1[i][j]=float(temp[i][j]/(2*sum))

def getAngularSecondMoment(CCMP1): # Угловой второй момент-Мера однородности изображения
  res=0
  for i in range (qlc):
    for j in range (qlc):
      res+=pow(CCMP1[i][j],2)
  return res


def getContrast(CCMP1): #Контрастность-Мера локальной вариации изображения
  res=0
  for i in range (qlc):
    for j in range (qlc):
      res+=(i-j)*(i-j)*CCMP1[i][j]
  return res


def Calc_Px(CCMP1,Px1):
  temp=[0]*qlc
  for i in range (qlc):
    for j in range (qlc):
      temp[j]+=CCMP1[i][j]
  for i in range(qlc):
    Px1[i]=temp[i]


def Calc_Py(CCMP1,Py1):
  temp=[0]*qlc
  for i in range (qlc):
    for j in range (qlc):
      temp[i]+=CCMP1[i][j]
  for i in range(qlc):
    Py1[i]=temp[i]


def Calc_M(P1):
  res=0
  for i in range(1,qlc):
    res+=i*P1[i]
  return res


def Calc_D(P1,M1):
  res=0
  for i in range(qlc):
    res+=(pow(i-M1,2)*P1[i])
  return math.sqrt(res)


def Calc_P_X_Plus_Y(CCMP1,PxPy1):
  temp=[0.0]*(qlc+qlc)
  for i in range (qlc):
    for j in range (qlc):
      temp[i+j]+=CCMP1[i][j]
  for i in range (qlc+qlc):
    PxPy1[i]=temp[i]


def Calc_P_X_Minus_Y(CCMP1,PxMy1):
  temp=[0.0]*(qlc+qlc)
  for i in range (qlc):
    for j in range (qlc):
      di=abs(i-j)
      temp[di]+=CCMP1[i][j]
  for i in range (qlc+qlc):
    PxMy1[i]=temp[i]


def getCorr(CCMP1,Mx1,My1,Dx1,Dy1): #Корреляция-Мера линейной зависимости значений интенсивности на изображении
  res=0
  for i in range (qlc):
    for j in range (qlc):
      res+=(i*j)*CCMP1[i][j]
  res-=(Mx1*My1)
  res/=(Dx1*Dy1)
  return res


def GetSumOfSquares(CCMP1, Mx1): # отклонение суммы квадратов
  res=0
  for i in range (qlc):
    for j in range (qlc):
      di=i-Mx1
      res+=di*di*CCMP1[i][j]
  return res


def GetInverseDifferenceMoment(CCMP1):  #обратный момент разности-Еще одна мера для представления контрастности изображения
  res=0
  for i in range (qlc):
    for j in range (qlc):
      di=i-j
      res+=CCMP1[i][j]/(1+di*di)
  return res


def GetSumAverage(PxPy1,qlc1): #среднее значение суммы-Среднее значение нормализованного изображения в оттенках серого в пространственной области.
  res=0
  for i in range (qlc1):
    if(PxPy1[i]>0):
      res+=i*PxPy1[i]
  return res+2


def GetSumEntropy(PxPy1,qlc1): # разность энтропий-Мера случайности внутри изображения.
  res=0
  for i in range (0,qlc1):
    if(PxPy1[i]>0):
      res+=PxPy1[i]*math.log10(PxPy1[i])
  return -res


def GetSumVariance(PxPy1,qlc1,avsum1): #Дисперсия нормализованного изображения в оттенках серого в пространственной области.
  res=0
  for i in range (0,qlc1):
    if(PxPy1[i]>0):
      res+=pow((i+1-avsum1),2)*PxPy1[i]
  return int(res)


def GetEntropy(CCMP1): #энтропия
  res=0
  for i in range (qlc):
    for j in range (qlc):
      if(CCMP1[i][j]>0):
        res+=CCMP1[i][j]*math.log10(CCMP1[i][j])
  return -res



#мб нужно вставить параметр (sum)PxMy/n)
def GetDiffereceVariance(PxMy1,qlc1,entsum1): #разность дисперсий
  res=0
  for i in range (0,qlc1):
    if(PxMy1[i]>0):
      res+=pow((i-entsum1),2)*PxMy1[i]
  return res


def GetDefferenceEntropy(PxMy1,qlc1): #разность энтропий
  res=0
  for i in range (qlc1):
    if(PxMy1[i]>0):
      res+=PxMy1[i]*math.log10(PxMy1[i])
  return -res

In [14]:
def CalcRunLengthFeaturesW(arr1,rows1,cols1,par,matr,rowsW1,colsW1,features1,start):
  calcRunLengthMatrixW(arr1, rows1, cols1, par, matr, rowsW1, colsW1)
  C=calC(matr,rowsW1,colsW1)
  features1[start+3]=calcShrtREmph(matr,rowsW1,colsW1,C) #Обратный момент
  features1[start+2]=calcLngREmph(matr,rowsW1,colsW1,C) #прямой момент
  features1[start+1]=calcGLevNonUni(matr,rowsW1,colsW1,C) #Неравномерность уровня серого
  features1[start]=calcRLNonUni(matr,rowsW1,colsW1,C) #Неравномерность длины пробега
  features1[start+4]=calcFraction(matr,rowsW1,colsW1,C) #доля изображения


def calcRunLengthMatrixW(arr2, rows2, cols2, par, matr, rowsH2, colsH2):
  di=1
  dj=0
  t0=0
  t1=cols2
  count=rows2
  for t in range(t0,t1):
    posI=0
    posJ=t
    runlen=0
    val=arr2[posI][posJ]
    lastValue = val
    runlen+=1
    posI+=di
    posJ+=dj
    for c in range(0,count-1):
      val=arr2[posI][posJ]
      if lastValue!=val:
        matr[lastValue][runlen-1]+=1
        runlen = 0;
      lastValue = val
      runlen+=1
      posI+=di
      posJ+=dj
    matr[lastValue][runlen-1]+=1


def calcRunLengthMatrixHight(arr2, rows2, cols2, par, matr, rowsH2, colsH2):
  di=0
  dj=1
  t0=0
  t1=rows2
  count=cols2
  for t in range(t0,t1):
    posI=t
    posJ=0
    runlen=0
    val=arr2[posI][posJ]
    lastValue = val
    runlen+=1
    posI+=di
    posJ+=dj
    for c in range(0,count-1):
      val=arr2[posI][posJ]
      if lastValue!=val:
        matr[lastValue][runlen-1]+=1
        runlen = 0;
      lastValue = val
      runlen+=1
      posI+=di
      posJ+=dj
    matr[lastValue][runlen-1]+=1


def CalcRunLengthFeaturesHight(arr1,rows1,cols1,par,matr,rowsH1,colsH1,features1,start):
  calcRunLengthMatrixHight(arr1, rows1, cols1, par, matr, rowsH1, colsH1)
  C=calC(matr,rowsH1,colsH1)
  features1[start+3]=calcShrtREmph(matr,rowsH1,colsH1,C)
  features1[start+2]=calcLngREmph(matr,rowsH1,colsH1,C)
  features1[start+1]=calcGLevNonUni(matr,rowsH1,colsH1,C)
  features1[start]=calcRLNonUni(matr,rowsH1,colsH1,C)
  features1[start+4]=calcFraction(matr,rowsH1,colsH1,C)


def calC(matr,rows11,cols11):
  res=0
  for i in range(rows11):
    for j in range (cols11):
      res+=matr[i][j]
  return res


def calcShrtREmph(matr,rows11,cols11,C):
  res=0
  for i in range(0,rows11):
    for j in range(0,cols11):
      res+=float(matr[i][j])/float((j+1)*(j+1))
  return (res/C)


def calcLngREmph(matr,rows11,cols11,C):
  res=0
  for i in range(0,rows11):
    for j in range(0,cols11):
      res+=float(matr[i][j])*((j+1)*(j+1))
  return (res/C)


def calcGLevNonUni(matr,rows11,cols11,C):
  res=0
  locres=0
  for i in range(0,rows11):
    locres=0
    for j in range(0,cols11):
      locres+=(matr[i][j])
    res+=pow(locres,2)
  return (res/C)


def calcRLNonUni(matr,rows11,cols11,C):
  res=0
  locres=0
  for j in range(0,cols11):
    locres=0
    for i in range(0,rows11):
      locres+=(matr[i][j])
    res+=pow(locres,2)
  return (res/C)


def calcFraction(matr,rows11,cols11,C):
  res=0
  for i in range(0,rows11):
    for j in range(0,cols11):
      res+=(matr[i][j])*(j+1)
  return (C/res)

In [15]:
def getAbsoluteGradient(arr1,rows1,cols1):
  gradRows=rows1-2
  gradCols=cols1-2
  res=[0]*gradRows
  for i in range(gradRows):
    res[i]=[0]*gradCols
  for a in range(1,rows1-1):
    for j in range(1,cols1-1):
      di=int(arr1[a+1][j])-int(arr1[a-1][j])
      dj=int(arr1[a][j+1])-int(arr1[a][j-1])
      res[a-1][j-1]=math.sqrt(di*di+dj*dj)
  return res

def getGradientFeatures(arr1,rows1,cols1,gr1,fearures1,start1):
  absgrmean=GetAbsoluteGradientMean(gr1,rows1-2,cols1-2)
  fearures1[start1]=absgrmean
  absgrsigma=GetAbsoluteGradientSigma(gr1,rows1-2,cols1-2,absgrmean)
  fearures1[start1+1]=absgrsigma
  absgrkurt=GetAbsoluteGradientKurtosis(gr1,rows1-2,cols1-2,absgrmean,absgrsigma)
  fearures1[start1+3]=absgrkurt
  absgrskwn=GetAbsoluteGradientSkewness(gr1,rows1-2,cols1-2,absgrmean,absgrsigma)
  fearures1[start1+2]=absgrskwn
  absgrnonz=GetAbsoluteGradientNonZeros(gr1,rows1-2,cols1-2)
  fearures1[start1+4]=absgrnonz

def GetAbsoluteGradientMean(gr1,rows1,cols1): #среднее значение
  res=0
  for i in range(1,rows1):
    for j in range(1,cols1):
      res+=gr1[i][j]
  return res/((rows1)*(cols1))

def GetAbsoluteGradientSigma(gr1,rows1,cols1,absgrmean1): #сигма градиента
  res=0
  for i in range(1,rows1):
    for j in range(1,cols1):
      res+=pow(gr1[i][j]-absgrmean1,2)
  return res/(rows1*cols1)

def GetAbsoluteGradientKurtosis(gr1,rows1,cols1,absgrmean1,absgrsigma1): #коэффициент эксцесса градиента
  res=0
  for i in range(1,rows1):
    for j in range(1,cols1):
      res+=pow(gr1[i][j]-absgrmean1,4)
  return ((res/(rows1*cols1*pow(absgrsigma1,2)))-3)

def GetAbsoluteGradientSkewness(gr1,rows1,cols1,absgrmean1,absgrsigma1): #коэффициент ассиметрии градиента
  res=0
  for i in range(1,rows1):
    for j in range(1,cols1):
      res+=pow(gr1[i][j]-absgrmean1,3)
  return (res/(rows1*cols1*pow(absgrsigma1,1.5)))

def GetAbsoluteGradientNonZeros(gr1,rows1,cols1): #количесвто ненулевые значений градиента
  res=0
  for i in range(1,rows1):
    for j in range(1,cols1):
      if (gr1[i][j]>0):
        res+=1
  return (res/(rows1*cols1))

In [16]:
def Haar_last(a,LL,LH,HL,HH,feat,start):
  wave1=0
  wave2=0
  wave3=0
  wave4=0
  for  i in range (0, LL.shape[0]):
    for  j in range (0, LL.shape[1]):
      wave1+=pow(LL[i][j],2)
      wave2+=pow(LH[i][j],2)
      wave3+=pow(HL[i][j],2)
      wave4+=pow(HH[i][j],2)
  feat[start]=wave1/(a)
  feat[start+1]=wave2/(a)
  feat[start+2]=wave3/(a)
  feat[start+3]=wave4/(a)

def Haar(a,LH,HL,HH,feat,start):
  wave2=0
  wave3=0
  wave4=0
  for  i in range (0, HH.shape[0]):
    for  j in range (0, HH.shape[1]):
      wave2+=pow(LH[i][j],2)
      wave3+=pow(HL[i][j],2)
      wave4+=pow(HH[i][j],2)
  feat[start+1]=wave2/(a)
  feat[start+2]=wave3/(a)
  feat[start+3]=wave4/(a)
  feat[start]=feat[start+4]+feat[start+5]+feat[start+6]+feat[start+7]

In [17]:
# um = '/content/umpriz.txt'
# um_mass = np.loadtxt(um)
# um_mass=um_mass.reshape(17,244)
# import pandas as pd
# df = pd.DataFrame(um_mass)
# df.to_excel(excel_writer = "test.xlsx")

In [18]:
# вывод признаков
for i in range(0,len(name_of_features)):
  print(name_of_features[i]+" = "+ str(features[i]) +"\n")

Mean = 0

Variance = 0

Skewness = 0

Kurtosis = 0

Perc.01% = 0

Perc.10% = 0

Perc.50% = 0

Perc.90% = 0

Perc.99% = 0

S(1,0)AngScMom = 0

S(1,0)Concat = 0

S(1,0)Correlat = 0

S(1,0)SumOfSqs = 0

S(1,0)InvDfMom = 0

S(1,0)SumAverg = 0

S(1,0)SumVarnc = 0

S(1,0)SumEntrp = 0

S(1,0)Entropy = 0

S(1,0)DifVarnc = 0

S(1,0)DifEntr = 0

S(0,1)AngScMom = 0

S(0,1)Concat = 0

S(0,1)Correlat = 0

S(0,1)SumOfSqs = 0

S(0,1)InvDfMom = 0

S(0,1)SumAverg = 0

S(0,1)SumVarnc = 0

S(0,1)SumEntrp = 0

S(0,1)Entropy = 0

S(0,1)DifVarnc = 0

S(0,1)DifEntr = 0

S(1,1)AngScMom = 0

S(1,1)Concat = 0

S(1,1)Correlat = 0

S(1,1)SumOfSqs = 0

S(1,1)InvDfMom = 0

S(1,1)SumAverg = 0

S(1,1)SumVarnc = 0

S(1,1)SumEntrp = 0

S(1,1)Entropy = 0

S(1,1)DifVarnc = 0

S(1,1)DifEntr = 0

S(1,-1)AngScMom = 0

S(1,-1)Concat = 0

S(1,-1)Correlat = 0

S(1,-1)SumOfSqs = 0

S(1,-1)InvDfMom = 0

S(1,-1)SumAverg = 0

S(1,-1)SumVarnc = 0

S(1,-1)SumEntrp = 0

S(1,-1)Entropy = 0

S(1,-1)DifVarnc = 0

S(1,-1)DifEntr = 0

S(2

IndexError: list index out of range

In [ ]:
#Haar Wavelet, с достраиванием изображения (вэйвлеты без достраивания считаюься полностью правильно, с достройкой есть больше ошибок)
arg1=1
while arg1 < rows:
  arg1 = arg1 << 1
arg2=1
while arg2 < cols:
  arg2 = arg2 << 1
if arg1>4096 or arg2>4096:
  print("cant calculate wavelet")
else:
  if (max(arg1,arg2)>16 and max(arg1,arg2)<=32):
    lev=1
    new_im = Getnewimage(imagegray,lev)
    cA1, (cH1, cV1, cD1) = wavedec2(new_im, 'haar', level=lev)
    Haar_last(arg1*arg2,cA1,cH1,cV1,cD1,features,244)
  if max(arg1,arg2)>32 and max(arg1,arg2)<=64:
    lev=2
    new_im = Getnewimage(imagegray,lev)
    cA2,(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(new_im, 'haar', level=lev)
    Haar_last(arg1*arg2,cA2,cH2,cV2,cD2,features,244)
    Haar(arg1*arg2,cH1,cV1,cD1,features,248)
  if max(arg1,arg2)>64 and max(arg1,arg2)<=128:
    lev=3
    new_im = Getnewimage(imagegray,lev)
    cA3,(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(new_im, 'haar', level=lev)
    Haar_last(arg1*arg2,cA3,cH3,cV3,cD3,features,244)
    Haar(arg1*arg2,cH2,cV2,cD2,features,248)
    Haar(arg1*arg2,cH1,cV1,cD1,features,252)
  if max(arg1,arg2)>128 and max(arg1,arg2)<=256:
    lev=4
    new_im = Getnewimage(imagegray,lev)
    cA4,(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(new_im, 'haar', level=lev)
    Haar_last(arg1*arg2,cA4,cH4,cV4,cD4,features,244)
    Haar(arg1*arg2,cH3,cV3,cD3,features,248)
    Haar(arg1*arg2,cH2,cV2,cD2,features,252)
    Haar(arg1*arg2,cH1,cV1,cD1,features,256)
  if max(arg1,arg2)>256 and max(arg1,arg2)<=512:
    lev=5
    new_im = Getnewimage(imagegray,lev)
    cA5,(cH5, cV5, cD5),(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(new_im, 'haar', level=lev)
    Haar_last(arg1*arg2,cA5,cH5,cV5,cD5,features,244)
    Haar(arg1*arg2,cH4, cV4, cD4,features,248)
    Haar(arg1*arg2,cH3,cV3,cD3,features,252)
    Haar(arg1*arg2,cH2,cV2,cD2,features,256)
    Haar(arg1*arg2,cH1,cV1,cD1,features,260)
  if max(arg1,arg2)>512 and max(arg1,arg2)<=1024:
    lev=6
    new_im = Getnewimage(imagegray,lev)
    cA6,(cH6, cV6, cD6),(cH5, cV5, cD5),(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(new_im, 'haar', level=lev)
    Haar_last(arg1*arg2,cA6,cH6, cV6, cD6,features,244)
    Haar(arg1*arg2,cH5, cV5, cD5,features,248)
    Haar(arg1*arg2,cH4, cV4, cD4,features,252)
    Haar(arg1*arg2,cH3,cV3,cD3,features,256)
    Haar(arg1*arg2,cH2,cV2,cD2,features,260)
    Haar(arg1*arg2,cH1,cV1,cD1,features,264)
  if max(arg1,arg2)>1024 and max(arg1,arg2)<=2048:
    lev=7
    new_im = Getnewimage(imagegray,lev)
    cA7,(cH7, cV7, cD7),(cH6, cV6, cD6),(cH5, cV5, cD5),(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(new_im, 'haar', level=lev)
    Haar_last(arg1*arg2,cA7,cH7, cV7, cD7,features,244)
    Haar(arg1*arg2,cH6, cV6, cD6,features,248)
    Haar(arg1*arg2,cH5, cV5, cD5,features,252)
    Haar(arg1*arg2,cH4, cV4, cD4,features,256)
    Haar(arg1*arg2,cH3,cV3,cD3,features,260)
    Haar(arg1*arg2,cH2,cV2,cD2,features,264)
    Haar(arg1*arg2,cH1,cV1,cD1,features,268)
  if max(arg1,arg2)>2048 and max(arg1,arg2)<=4096:
    lev=8
    new_im = Getnewimage(imagegray,lev)
    cA8,(cH8, cV8, cD8),(cH7, cV7, cD7),(cH6, cV6, cD6),(cH5, cV5, cD5),(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(new_im, 'haar', level=lev)
    Haar_last(arg1*arg2,cA8,cH8, cV8, cD8,features,244)
    Haar(arg1*arg2,cH7, cV7, cD7,features,248)
    Haar(arg1*arg2,cH6, cV6, cD6,features,252)
    Haar(arg1*arg2,cH5, cV5, cD5,features,256)
    Haar(arg1*arg2,cH4, cV4, cD4,features,260)
    Haar(arg1*arg2,cH3,cV3,cD3,features,264)
    Haar(arg1*arg2,cH2,cV2,cD2,features,268)
    Haar(arg1*arg2,cH1,cV1,cD1,features,272)

NameError: name 'Getnewimage' is not defined

In [ ]:
def Haar(a,LH,HL,HH,feat,start):
  wave2=0
  wave3=0
  wave4=0
  for  i in range (0, HH.shape[0]):
    for  j in range (0, HH.shape[1]):
      wave2+=pow(LH[i][j],2)
      wave3+=pow(HL[i][j],2)
      wave4+=pow(HH[i][j],2)
  feat[start+1]=wave2/(a)
  feat[start+2]=wave3/(a)
  feat[start+3]=wave4/(a)
  feat[start]=feat[start-4]-feat[start+1]-feat[start+2]-feat[start+3]

In [ ]:
def Getnewimage(image,level):
  perem=pow(2,4+level)
  imagegray1=imagegray.crop((0,0,rows,perem-cols))
  imagegraywave=Image.new('L', (perem,perem))
  imagegraywave.paste(imagegray,(0,0))
  imagegraywave.paste(imagegray1,(0,rows))
  imagegraywave1=imagegraywave.crop((0,0,perem-cols,perem))
  imagegraywave.paste(imagegraywave1,(cols,0))
  return imagegraywave

In [ ]:
#Haar Wavelet без достройки
arg1=1
while arg1 < rows:
  arg1 = arg1 << 1
arg2=1
while arg2 < cols:
  arg2 = arg2 << 1
if arg1>4096 or arg2>4096:
  print("cant calculate wavelet")
else:
  if arg1>16 and arg1<=32 and arg2>16 and arg2<=32:
    lev=1
    cA1, (cH1, cV1, cD1) = wavedec2(imagegray, 'haar', level=lev)
    Haar_last(arg1*arg2,cA1,cH1,cV1,cD1,features,244)
  if arg1>32 and arg1<=64 and arg2>32 and arg2<=64:
    lev=2
    cA2,(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(imagegray, 'haar', level=lev)
    Haar_last(arg1*arg2,cA2,cH2,cV2,cD2,features,244)
    Haar(arg1*arg2,cH1,cV1,cD1,features,248)
  if arg1>64 and arg1<=128 and arg2>64 and arg2<=128:
    lev=3
    cA3,(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(imagegray, 'haar', level=lev)
    Haar_last(arg1*arg2,cA3,cH3,cV3,cD3,features,244)
    Haar(arg1*arg2,cH2,cV2,cD2,features,248)
    Haar(arg1*arg2,cH1,cV1,cD1,features,252)
  if arg1>128 and arg1<=256 and arg2>128 and arg2<=256:
    lev=4
    cA4,(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(imagegray, 'haar', level=lev)
    Haar_last(arg1*arg2,cA4,cH4,cV4,cD4,features,244)
    Haar(arg1*arg2,cH3,cV3,cD3,features,248)
    Haar(arg1*arg2,cH2,cV2,cD2,features,252)
    Haar(arg1*arg2,cH1,cV1,cD1,features,256)
  if arg1>256 and arg1<=512 and arg2>256 and arg2<=512:
    lev=5
    cA5,(cH5, cV5, cD5),(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(imagegray, 'haar', level=lev)
    Haar_last(arg1*arg2,cA5,cH5,cV5,cD5,features,244)
    Haar(arg1*arg2,cH4, cV4, cD4,features,248)
    Haar(arg1*arg2,cH3,cV3,cD3,features,252)
    Haar(arg1*arg2,cH2,cV2,cD2,features,256)
    Haar(arg1*arg2,cH1,cV1,cD1,features,260)
  if arg1>512 and arg1<=1024 and arg2>512 and arg2<=1024:
    lev=6
    cA6,(cH6, cV6, cD6),(cH5, cV5, cD5),(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(imagegray, 'haar', level=lev)
    Haar_last(arg1*arg2,cA6,cH6, cV6, cD6,features,244)
    Haar(arg1*arg2,cH5, cV5, cD5,features,248)
    Haar(arg1*arg2,cH4, cV4, cD4,features,252)
    Haar(arg1*arg2,cH3,cV3,cD3,features,256)
    Haar(arg1*arg2,cH2,cV2,cD2,features,260)
    Haar(arg1*arg2,cH1,cV1,cD1,features,264)
  if arg1>1024 and arg1<=2048 and arg2>1024 and arg2<=2048:
    lev=7
    cA7,(cH7, cV7, cD7),(cH6, cV6, cD6),(cH5, cV5, cD5),(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(imagegray, 'haar', level=lev)
    Haar_last(arg1*arg2,cA7,cH7, cV7, cD7,features,244)
    Haar(arg1*arg2,cH6, cV6, cD6,features,248)
    Haar(arg1*arg2,cH5, cV5, cD5,features,252)
    Haar(arg1*arg2,cH4, cV4, cD4,features,256)
    Haar(arg1*arg2,cH3,cV3,cD3,features,260)
    Haar(arg1*arg2,cH2,cV2,cD2,features,264)
    Haar(arg1*arg2,cH1,cV1,cD1,features,268)
  if arg1>2048 and arg1<=4096 and arg2>2048 and arg2<=4096:
    lev=8
    cA8,(cH8, cV8, cD8),(cH7, cV7, cD7),(cH6, cV6, cD6),(cH5, cV5, cD5),(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(imagegray, 'haar', level=lev)
    Haar_last(arg1*arg2,cA8,cH8, cV8, cD8,features,244)
    Haar(arg1*arg2,cH7, cV7, cD7,features,248)
    Haar(arg1*arg2,cH6, cV6, cD6,features,252)
    Haar(arg1*arg2,cH5, cV5, cD5,features,256)
    Haar(arg1*arg2,cH4, cV4, cD4,features,260)
    Haar(arg1*arg2,cH3,cV3,cD3,features,264)
    Haar(arg1*arg2,cH2,cV2,cD2,features,268)
    Haar(arg1*arg2,cH1,cV1,cD1,features,272)

NameError: name 'wavedec2' is not defined

In [ ]:
import glob
images_path = "masked/"
paths = os.listdir(images_path)
print(paths)
for c, exam_path in enumerate(paths):
  image = Image.open(images_path+exam_path)
  imagegray = ImageOps.grayscale(image)
  cols,rows = image.size
  imagegray.transpose(PIL.Image.ROTATE_90)
  arr=np.asarray(imagegray,dtype='uint8')
  hist=[0]*256
  raz=256
  features=[0]*244 #количество признаков, 244 без вэйвлет, а с ними на 264
  qlc=256
  CCMP = [0]*qlc
  for i in range(qlc):
    CCMP[i]=[0]*qlc
  Px=[0.0]*qlc
  Py=[0.0]*qlc
  PxPy=[0.0]*(qlc+qlc)
  PxMy=[0.0]*(qlc+qlc)
  quantization_div = int(256/qlc)
  HEIGHT=0
  WIDTH=1
  ANGLE45=2
  ANGLE135=3

#названия признаков (с вэйвлетами 5 уровня,если без них, то последние 20 убрать)
  name_of_features=['Mean','Variance','Skewness','Kurtosis','Perc.01%','Perc.10%','Perc.50%','Perc.90%','Perc.99%','S(1,0)AngScMom','S(1,0)Concat','S(1,0)Correlat',
                  'S(1,0)SumOfSqs','S(1,0)InvDfMom','S(1,0)SumAverg','S(1,0)SumVarnc','S(1,0)SumEntrp','S(1,0)Entropy','S(1,0)DifVarnc','S(1,0)DifEntr','S(0,1)AngScMom',
                  'S(0,1)Concat','S(0,1)Correlat','S(0,1)SumOfSqs','S(0,1)InvDfMom','S(0,1)SumAverg','S(0,1)SumVarnc','S(0,1)SumEntrp','S(0,1)Entropy','S(0,1)DifVarnc',
                  'S(0,1)DifEntr','S(1,1)AngScMom','S(1,1)Concat','S(1,1)Correlat','S(1,1)SumOfSqs','S(1,1)InvDfMom','S(1,1)SumAverg','S(1,1)SumVarnc','S(1,1)SumEntrp',
                  'S(1,1)Entropy','S(1,1)DifVarnc','S(1,1)DifEntr','S(1,-1)AngScMom','S(1,-1)Concat','S(1,-1)Correlat','S(1,-1)SumOfSqs','S(1,-1)InvDfMom','S(1,-1)SumAverg',
                  'S(1,-1)SumVarnc','S(1,-1)SumEntrp','S(1,-1)Entropy','S(1,-1)DifVarnc','S(1,-1)DifEntr','S(2,0)AngScMom','S(2,0)Concat','S(2,0)Correlat',
                  'S(2,0)SumOfSqs','S(2,0)InvDfMom','S(2,0)SumAverg','S(2,0)SumVarnc','S(2,0)SumEntrp','S(2,0)Entropy','S(2,0)DifVarnc','S(2,0)DifEntr','S(0,2)AngScMom',
                  'S(0,2)Concat','S(0,2)Correlat','S(0,2)SumOfSqs','S(0,2)InvDfMom','S(0,2)SumAverg','S(0,2)SumVarnc','S(0,2)SumEntrp','S(0,2)Entropy','S(0,2)DifVarnc',
                  'S(0,2)DifEntr','S(2,2)AngScMom','S(2,2)Concat','S(2,2)Correlat','S(2,2)SumOfSqs','S(2,2)InvDfMom','S(2,2)SumAverg','S(2,2)SumVarnc','S(2,2)SumEntrp',
                  'S(2,2)Entropy','S(2,2)DifVarnc','S(2,2)DifEntr','S(2,-2)AngScMom','S(2,-2)Concat','S(2,-2)Correlat','S(2,-2)SumOfSqs','S(2,-2)InvDfMom','S(2,-2)SumAverg',
                  'S(2,-2)SumVarnc','S(2,-2)SumEntrp','S(2,-2)Entropy','S(2,-2)DifVarnc','S(2,-2)DifEntr','S(3,0)AngScMom','S(3,0)Concat','S(3,0)Correlat',
                  'S(3,0)SumOfSqs','S(3,0)InvDfMom','S(3,0)SumAverg','S(3,0)SumVarnc','S(3,0)SumEntrp','S(3,0)Entropy','S(3,0)DifVarnc','S(3,0)DifEntr','S(0,3)AngScMom',
                  'S(0,3)Concat','S(0,3)Correlat','S(0,3)SumOfSqs','S(0,3)InvDfMom','S(0,3)SumAverg','S(0,3)SumVarnc','S(0,3)SumEntrp','S(0,3)Entropy','S(0,3)DifVarnc',
                  'S(0,3)DifEntr','S(3,3)AngScMom','S(3,3)Concat','S(3,3)Correlat','S(3,3)SumOfSqs','S(3,3)InvDfMom','S(3,3)SumAverg','S(3,3)SumVarnc','S(3,3)SumEntrp',
                  'S(3,3)Entropy','S(3,3)DifVarnc','S(3,3)DifEntr','S(3,-3)AngScMom','S(3,-3)Concat','S(3,-3)Correlat','S(3,-3)SumOfSqs','S(3,-3)InvDfMom','S(3,-3)SumAverg',
                  'S(3,-3)SumVarnc','S(3,-3)SumEntrp','S(3,-3)Entropy','S(3,-3)DifVarnc','S(3,-3)DifEntr','S(4,0)AngScMom','S(4,0)Concat','S(4,0)Correlat',
                  'S(4,0)SumOfSqs','S(4,0)InvDfMom','S(4,0)SumAverg','S(4,0)SumVarnc','S(4,0)SumEntrp','S(4,0)Entropy','S(4,0)DifVarnc','S(4,0)DifEntr','S(0,4)AngScMom',
                  'S(0,4)Concat','S(0,4)Correlat','S(0,4)SumOfSqs','S(0,4)InvDfMom','S(0,4)SumAverg','S(0,4)SumVarnc','S(0,4)SumEntrp','S(0,4)Entropy','S(0,4)DifVarnc',
                  'S(0,4)DifEntr','S(4,4)AngScMom','S(4,4)Concat','S(4,4)Correlat','S(4,4)SumOfSqs','S(4,4)InvDfMom','S(4,4)SumAverg','S(4,4)SumVarnc','S(4,4)SumEntrp',
                  'S(4,4)Entropy','S(4,4)DifVarnc','S(4,4)DifEntr','S(4,-4)AngScMom','S(4,-4)Concat','S(4,-4)Correlat','S(4,-4)SumOfSqs','S(4,-4)InvDfMom','S(4,-4)SumAverg',
                  'S(4,-4)SumVarnc','S(4,-4)SumEntrp','S(4,-4)Entropy','S(4,-4)DifVarnc','S(4,-4)DifEntr','S(5,0)AngScMom','S(5,0)Concat','S(5,0)Correlat',
                  'S(5,0)SumOfSqs','S(5,0)InvDfMom','S(5,0)SumAverg','S(5,0)SumVarnc','S(5,0)SumEntrp','S(5,0)Entropy','S(5,0)DifVarnc','S(5,0)DifEntr','S(0,5)AngScMom',
                  'S(0,5)Concat','S(0,5)Correlat','S(0,5)SumOfSqs','S(0,5)InvDfMom','S(0,5)SumAverg','S(0,5)SumVarnc','S(0,5)SumEntrp','S(0,5)Entropy','S(0,5)DifVarnc',
                  'S(0,5)DifEntr','S(5,5)AngScMom','S(5,5)Concat','S(5,5)Correlat','S(5,5)SumOfSqs','S(5,5)InvDfMom','S(5,5)SumAverg','S(5,5)SumVarnc','S(5,5)SumEntrp',
                  'S(5,5)Entropy','S(5,5)DifVarnc','S(5,5)DifEntr','S(5,-5)AngScMom','S(5,-5)Concat','S(5,-5)Correlat','S(5,-5)SumOfSqs','S(5,-5)InvDfMom','S(5,-5)SumAverg',
                  'S(5,-5)SumVarnc','S(5,-5)SumEntrp','S(5,-5)Entropy','S(5,-5)DifVarnc','S(5,-5)DifEntr','Horzl_RLNonUni','Horzl_GlevNonU','Horzl_LngREmph','Horzl_ShrtREmph',
                  'Horzl_Fraction','Vertl_RLNonUni','Vertl_GlevNonU','Vertl_LngREmph','Vertl_ShrtREmph','Vertl_Fraction','GrMean','GrVariance','GrSkewness','GrKurtosis','GrNonZeros',
                  'WavEnLL_s-1','WavEnLH_s-1','WavEnHL_s-1','WavEnHH_s-1','WavEnLL_s-2','WavEnLH_s-2','WavEnHL_s-2','WavEnHH_s-2','WavEnLL_s-3','WavEnLH_s-3','WavEnHL_s-3','WavEnHH_s-3',
                  'WavEnLL_s-4','WavEnLH_s-4','WavEnHL_s-4','WavEnHH_s-4','WavEnLL_s-5','WavEnLH_s-5','WavEnHL_s-5','WavEnHH_s-5']
  #GetHist получение гистограммы изображнения (сумма всех значений гистограммы равна 1)
  for i in range (rows):
    for j in range (cols):
      hist[arr[i][j]]+=1
  for i in range (raz):
    hist[i]/=(cols*rows)
  # гистограммные признаки изображения, расчитываются по гистаграмме
  start=0
  mean=getMean(hist,raz)
  variance=getVariance(hist,raz,mean)
  skewness=getSkewness(hist,raz,mean,variance)
  kurtosis=getKurtosis(hist,raz,mean,variance)
  perc01=getPerc(hist,raz,0.01)
  perc10=getPerc(hist,raz,0.10)
  perc50=getPerc(hist,raz,0.50)
  perc90=getPerc(hist,raz,0.90)
  perc99=getPerc(hist,raz,0.99)
  features[start]=mean
  features[start+1]=variance
  features[start+2]=skewness
  features[start+3]=kurtosis
  features[start+4]=perc01
  features[start+5]=perc10
  features[start+6]=perc50
  features[start+7]=perc90
  features[start+8]=perc99


#признаки Харалика, используют матрицу совпадений
#Матрица совпадений представляет собой двумерный массив, количество строк и столбцов в котором равно допустимому значению пикселей изобра-жения. В случае изображения в оттенках серого это значение равно 256. Зна-чение элемента матрицы с индексом i,j указывает, сколько раз значение i встречалось на изображении в некотором заданном пространственном отно-шении со значением j

#Матрица совпадений представляет собой двумерный массив, количество строк и столбцов в котором равно допустимому значению пикселей изображения.
#В случае изображения в оттенках серого это значение равно 256. Значение элемента матрицы с индексом i,j указывает, сколько раз значение i
# встречалось на изображении в некотором заданном пространственном отношении со значением j

  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,1,0,features,9)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,0,1,features,20)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,1,1,features,31)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,1,-1,features,42)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,2,0,features,53)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,0,2,features,64)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,2,2,features,75)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,2,-2,features,86)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,3,0,features,97)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,0,3,features,108)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,3,3,features,119)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,3,-3,features,130)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,4,0,features,141)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,0,4,features,152)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,4,4,features,163)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,4,-4,features,174)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,5,0,features,185)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,0,5,features,196)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,5,5,features,207)
  getHaralickFeatures(arr,rows,cols,CCMP,Px,Py,PxPy,PxMy,5,-5,features,218)



  # Матрица длины пробега основана на вероятностях пробега пикселей каждой возможной длины,
# расположенных в определенном направлении. Рассматриваются два стандартных направления прохождения пикселей, θ = 0◦ и 90◦.
# Матрица длины пробега, R(θ), содержит G столбцов и M строк, где G - количество уровней серого изображения,
# а M - максимальная длина пробега пикселя, которая может существовать в анализируемой области изображения.
  colors=265
  rowsH=colors
  colsH=cols
  matH = [0]*rowsH
  for i in range(rowsH):
    matH[i]=[0]*colsH

  colors=265
  rowsW=colors
  colsW=rows
  matW = [0]*rowsW
  for i in range(rowsW):
    matW[i]=[0]*colsW

  CalcRunLengthFeaturesHight(arr,rows,cols,HEIGHT,matH,rowsH,colsH,features,229)
  CalcRunLengthFeaturesW(arr,rows,cols,HEIGHT,matW,rowsW,colsW,features,234)


  #градиентные признаки
#Используем матрицу абсолютного градиента изображения. Для каждого пикселя, кроме краевых, находим градиент: рассчитываем параметры di и dj,
#равные разнице двух соседних пикселей: для di пиксели берутся по вертикальной ось, для dj пиксели берутся по горизонтальной,
#градиент же рассчитывается как корень из суммы квадратов di и dj
  otvn=getAbsoluteGradient(arr,rows,cols) # матрица абсолютного градиента изображения
  getGradientFeatures(arr,rows,cols,otvn,features,239)


  break
  #lev=5  # все изображения размером 512 на 512 пикселей
  #cA5,(cH5, cV5, cD5),(cH4, cV4, cD4),(cH3, cV3, cD3),(cH2, cV2, cD2), (cH1, cV1, cD1) = wavedec2(arr, 'haar', level=lev)
  #Haar_last(cols*rows,cA5,cH5,cV5,cD5,features,260)
  #Haar(cols*rows,cH4, cV4, cD4,features,256)
  #Haar(cols*rows,cH3,cV3,cD3,features,252)
  #Haar(cols*rows,cH2,cV2,cD2,features,248)
  #Haar(cols*rows,cH1,cV1,cD1,features,244)